In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import os
import glob
import re

In [2]:
#Constant variables
month = {
                'Jan' : '01',
                'Feb' : '02',
                'Mar' : '03',
                'May' : '04',
                'Apr' : '05',
                'Jun' : '06',
                'Jul' : '07',
                'Aug' : '08',
                'Sep' : '09',
                'Oct' : '10',
                'Nov' : '11',
                'Dec' : '12',  
}

# Use 2016-2019 Data to predict 2020's Gold Price

## import dataset

In [4]:
gold =pd.read_csv("./dataset/gld_price_data.csv")
gas =pd.read_csv("./dataset/natural_gas_data.csv")
oil = pd.read_csv('./dataset/crude_oil_eu_data.csv')
dolla = pd.read_csv('./dataset/dolla_index_data.csv')
etf = pd.read_csv('./dataset/etf_data.csv')
sil = pd.DataFrame(pd.read_csv('./dataset/silver_price_data.csv'), columns={'Date','USD'})
bit = pd.DataFrame(pd.read_csv('./dataset/bit_data.csv'), columns={'Date','Close'})

In [5]:
sil = sil[['Date','USD']]
bit = bit[['Date','Close']]

## data preprocessing

In [8]:
### daily data only
### df : list of dataframe , name : list of dataframe's value's name , color : list of data's color
df = [gold,gas,oil,dolla,etf,sil,bit]
columnName = ['gld_price', 'gas_price', 'oil_price','dolla_index','etf_index','silver_price','bitcoin_price']
color = ['red', 'green', 'blue', 'violet','purple','silver','gold']

In [9]:
### rename all data columns
for i in range(len(df)):
    df[i] = df[i].rename(columns={df[i].columns[0]:'date',df[i].columns[1]:columnName[i]})

In [10]:
### set date as index and join all data
data = df[0].set_index('date')
for i in range(len(df)):
    if i > 0:
        data = data.join(df[i].set_index('date'))

In [11]:
data

,gld_price,gas_price,oil_price,dolla_index,etf_index,silver_price,bitcoin_price
date,,,,,,,
2016-02-19,1231.15,1.83,31.66,113.9502,24.88,15.37,420.785004
2016-02-19,1267.00,1.83,31.66,113.9502,24.88,15.37,420.785004
2016-02-22,1211.00,1.86,33.59,113.8157,23.31,15.00,437.747986
2016-02-23,1221.35,1.85,31.90,114.1686,23.69,15.21,420.735992
2016-02-24,1250.75,1.85,31.50,114.163,24.24,15.32,424.954987
2016-02-25,1236.00,1.78,32.83,113.916,23.36,15.16,424.544006
2016-02-26,1226.50,1.78,35.76,114.365,21.67,15.17,432.152008
2016-02-29,1234.90,1.62,35.92,114.2359,22.59,14.75,437.696991
2016-03-01,1236.50,1.57,35.73,113.9208,21.97,14.99,435.122986


In [ ]:
data = data.replace('.', np.NaN)
data = data.dropna()

In [ ]:
### convert index to datetime64
data = pd.DataFrame(data, index = np.array([np.datetime64(i) for i in data.index]))

In [ ]:
### convert all data values to float
for i in data.columns:
    data[i] = data[i].astype(str).astype(float)